In [ ]:
import pandas as pd

In [ ]:
lib_id = "pluskal_nih"

In [ ]:
df1 = pd.read_csv("data/library/PNNL_GNPS_NIH_Library_Plate1_28.csv")
df2 = pd.read_csv("data/library/PNNL_GNPS_NIH_Library_Plate29_53.csv")
df = pd.concat([df1, df2], ignore_index=True)
df

In [ ]:
df["Platenumber"] = [int(plate_id.split("_")[1]) for plate_id in df["lib_plate_well"]]
df[["WellNumber", "WellLetter"]] = [[int(value[1:]),value[:1]] for value in df["well"]]
sorted_df = df.sort_values(by=["Platenumber", "WellLetter", "WellNumber"], ascending=[True, True, True])
sorted_df

In [ ]:
plate_dict = {
    1: 1,
    2: 1,
    3: 1,
    4: 1,
    5: 2,
    6: 2,
    7: 2,
    8: 2,
    9: 3,
    10: 3,
    11: 4,
    12: 4,
    13: 4,
    14: 4,
    15: 5,
    16: 5,
    17: 5,
    18: 5,
    19: 6,
    20: 6,
    21: 6,
    22: 6,
    23: 7,
    24: 7,
    25: 7,
    26: 7,
    27: 8,
    28: 8,
    29: 9,
    30: 9,
    31: 9,
    32: 9,
    33: 10,
    34: 10,
    35: 10,
    36: 10,
    37: 11,
    38: 12,
    39: 12,
    40: 12,
    41: 12,
    42: 13,
    43: 13,
    44: 13,
    45: 13,
    46: 14,
    47: 14,
    48: 14,
    49: 14,
    50: 15,
    51: 15,
    52: 15,
    53: 15

}

sorted_df["mixed_plate"] = [int(plate_dict.get(plate)) for plate in sorted_df['Platenumber']]

In [ ]:
sorted_df["mixed_location_plate"] = ["{:02d}P_{}{}".format(plate, letter, well) for plate, letter, well in zip(sorted_df["mixed_plate"], sorted_df["WellLetter"], sorted_df["WellNumber"])]

In [ ]:
sorted_df["lib_plate_well"] =  ["{}_{}".format(lib_id, plate_id) for plate_id in sorted_df["mixed_location_plate"]]

In [ ]:
sorted_df = sorted_df.sort_values(by=["mixed_plate", "WellLetter", "WellNumber"], ascending=[True, True, True])

In [ ]:
sorted_df

In [ ]:
sorted_df.to_csv("data/nih/nih_library.tsv", sep="\t", index=False)